In [1]:
!pip install --upgrade kagglehub[pandas-datasets,hf-datasets]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 7.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: kagglehub
    Found existing installation: kagglehub 0.3.11
    Uninstalling kagglehub-0.3.11:
      Successfully uninstalled kagglehub-0.3.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.8.4.1 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cudnn-cu12==9.1.0.70; plat

In [2]:
!apt-get install git -y

!git clone https://github.com/miriam-16/aml_1_aerial_imagery.git

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.12).
0 upgraded, 0 newly installed, 0 to remove and 122 not upgraded.
fatal: destination path 'aml_1_aerial_imagery' already exists and is not an empty directory.


In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import copy
import os
import torch
from PIL import Image
from PIL import Image, ImageDraw
from torch.utils.data import Dataset
import torchvision.transforms as transforms
from torch.utils.data import random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn as nn
from torchvision import utils
%matplotlib inline

In [4]:
path = "/kaggle/working/aml_1_aerial_imagery/dataset"
import pandas as pd

labels_df = pd.read_csv(path+'/train.csv')  # Adjust filename
print(labels_df.head())


                                     id  has_cactus
0  0004be2cfeaba1c0361d39e2b000257b.jpg           1
1  000c8a36845c0208e833c79c1bffedd1.jpg           1
2  000d1e9a533f62e55c289303b072733d.jpg           1
3  0011485b40695e9138e92d0b3fb55128.jpg           1
4  0014d7a11e90b62848904c1418fc8cf2.jpg           1


In [5]:
print(labels_df.shape)
labels_df[labels_df.duplicated(keep=False)]


(17500, 2)


,id,has_cactus


In [6]:
labels_df['has_cactus'].value_counts()

has_cactus
1    13136
0     4364
Name: count, dtype: int64

In [7]:
import torch
torch.manual_seed(0)

from torch.utils.data import Dataset
import os
import pandas as pd
from PIL import Image
import torchvision.transforms as transforms

class pytorch_data(Dataset):
    
    def __init__(self, data_dir, transform, data_type="train"):
        # Get Image File Names
        cdm_data = os.path.join(data_dir, data_type)
        file_names = os.listdir(cdm_data)

        all_image_paths = [os.path.join(cdm_data, f) for f in file_names if f.lower().endswith(('.jpg', '.png', '.jpeg'))]

        print(f"Found {len(all_image_paths)} images in directory.")
        print(f"Sample filenames: {all_image_paths[:1]}")

        # Get Labels
        labels_data = os.path.join("/kaggle/working/aml_1_aerial_imagery/dataset/", "train.csv")
        labels_df = pd.read_csv(labels_data)

        # Normalize index: remove extensions if present
        labels_df['id'] = labels_df['id'].apply(lambda x: os.path.splitext(str(x))[0])
        labels_df.set_index("id", inplace=True)

        print(f"Labels dataframe length: {len(labels_df)}")

        # Extract only valid images (that have a label)
        valid_filenames = []
        labels = []

        for f in all_image_paths:
            filename = os.path.basename(f)  # get only file name
            image_id = os.path.splitext(filename)[0]  # remove extension

            if image_id in labels_df.index:
                valid_filenames.append(f)
                labels.append(labels_df.loc[image_id].values[0])
            else:
                print(f"Warning: image '{filename}' has no matching label in train.csv")

        self.full_filenames = valid_filenames
        self.labels = labels
        self.transform = transform

        print(f"Valid image-label pairs: {len(self.full_filenames)}")
        print(f"First few labels: {self.labels[:5]}")
      
    def __len__(self):
        return len(self.full_filenames)
      
    def __getitem__(self, idx):
        if idx >= len(self.full_filenames):
            raise IndexError(f"Index {idx} out of bounds for dataset of length {len(self.full_filenames)}")

        image = Image.open(self.full_filenames[idx])
        image = self.transform(image)
        return image, self.labels[idx]

In [8]:
# define transformation that converts a PIL image into PyTorch tensors
import torchvision.transforms as transforms
data_transformer = transforms.Compose([transforms.ToTensor(),
                                       transforms.Resize((32,32))])


In [9]:
# Define an object of the custom dataset for the train folder.
data_dir = path+'/train/'
img_dataset = pytorch_data(data_dir, data_transformer, "train")

Found 17500 images in directory.
Sample filenames: ['/kaggle/working/aml_1_aerial_imagery/dataset/train/train/060a884c1005baed62bd6c4a72220da2.jpg']
Labels dataframe length: 17500
Valid image-label pairs: 17500
First few labels: [0, 0, 0, 0, 1]


In [10]:
# define transformation that converts a PIL image into PyTorch tensors
data_transformer = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((32, 32))
])

In [11]:
# Test a sample
img, label = img_dataset[10]
print(img.shape, torch.min(img), torch.max(img))

torch.Size([3, 32, 32]) tensor(0.1294) tensor(0.9412)


In [12]:
# Aumentiamo le immagini della classe 0
from torchvision.transforms import RandomRotation, ToTensor, Resize
from tqdm import tqdm

# Trasformazione per augmentare
augment_transform = transforms.Compose([
    RandomRotation(degrees=10),
    Resize((32, 32)),
    ToTensor()
])

# Trova solo immagini con etichetta 0
images_class0 = [i for i in range(len(img_dataset)) if img_dataset.labels[i] == 0]

# Duplichiamo queste immagini con trasformazione
augmented_images = []
augmented_labels = []

for idx in tqdm(images_class0):
    img_path = img_dataset.full_filenames[idx]
    img = Image.open(img_path)
    augmented_img = augment_transform(img)
    augmented_images.append(augmented_img)
    augmented_labels.append(0)

# Stack immagini originali
original_images = [img_dataset[i][0] for i in range(len(img_dataset))]
original_labels = [img_dataset[i][1] for i in range(len(img_dataset))]

# Combina immagini originali + augmentate
all_images = torch.stack(original_images + augmented_images)
all_labels = torch.tensor(original_labels + augmented_labels)

# Nuovo Dataset custom con dati augmentati
class AugmentedDataset(torch.utils.data.Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels
    def __len__(self):
        return len(self.images)
    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

# Sostituisci img_dataset con quello nuovo
img_dataset = AugmentedDataset(all_images, all_labels)

100%|██████████| 4364/4364 [00:01<00:00, 2354.59it/s]


In [14]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset

# Estrai le etichette in modo sicuro
labels = img_dataset.labels  # NON usare img_dataset[i][1]

# Crea una lista di tutti gli indici
all_indices = list(range(len(img_dataset)))

# Split stratificato: Train (70%) e Temp (30%)
train_idx, temp_idx = train_test_split(
    all_indices, test_size=0.3, stratify=labels, random_state=42
)

# Estrai le label corrispondenti agli indici temporanei per secondo split
temp_labels = [labels[i] for i in temp_idx]

# Split stratificato: Validation (15%) e Test (15%) da temp
val_idx, test_idx = train_test_split(
    temp_idx, test_size=0.5, stratify=temp_labels, random_state=42
)

# Crea i subset PyTorch
train_ts = Subset(img_dataset, train_idx)
val_ts = Subset(img_dataset, val_idx)
test_ts = Subset(img_dataset, test_idx)

# Visualizzazione
print("train dataset size:", len(train_ts))
print("validation dataset size:", len(val_ts))
print("test dataset size:", len(test_ts))

train dataset size: 15304
validation dataset size: 3280
test dataset size: 3280


# NEW MODEL: SVM

In [15]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import accuracy_score
import numpy as np
import torch

# Reuse conversion function
def dataset_to_numpy(dataset):
    X = []
    y = []
    for img, label in dataset:
        X.append(img.view(-1).numpy())  # Flatten image
        y.append(label)
    return np.array(X), np.array(y)

# Prepare data
X_train, y_train = dataset_to_numpy(train_ts)
X_val, y_val = dataset_to_numpy(val_ts)
X_test, y_test = dataset_to_numpy(test_ts)

# Normalize
X_train = X_train / 255.0
X_val = X_val / 255.0
X_test = X_test / 255.0

# Combine train and val for tuning
X_combined = np.concatenate((X_train, X_val))
y_combined = np.concatenate((y_train, y_val))


# Define parameter grid
C_values = [0.1, 1, 10]
kernels = ['rbf', 'sigmoid', 'poly']
gammas = ['scale', 'auto']

# Track best model
best_acc = 0
best_params = None
best_model = None

# Manual loop over all combinations
for C in C_values:
    for kernel in kernels:
        for gamma in gammas:
            model = SVC(C=C, kernel=kernel, gamma=gamma)
            model.fit(X_train, y_train)  # train ONLY on training set
            val_preds = model.predict(X_val)
            acc = accuracy_score(y_val, val_preds)
            print(f"Params: C={C}, kernel={kernel}, gamma={gamma} → Val Accuracy: {acc:.4f}")
            
            if acc > best_acc:
                best_acc = acc
                best_params = (C, kernel, gamma)
                best_model = model

print(f"\n✅ Best Params: C={best_params[0]}, kernel={best_params[1]}, gamma={best_params[2]}")
print(f"📈 Best Validation Accuracy: {best_acc:.2%}")

# Evaluate best model on test set
test_preds = best_model.predict(X_test)
test_acc = accuracy_score(y_test, test_preds)
print(f"\n🎯 Final Test Accuracy: {test_acc * 100:.2f}%")


Params: C=0.1, kernel=rbf, gamma=scale → Val Accuracy: 0.9210
Params: C=0.1, kernel=rbf, gamma=auto → Val Accuracy: 0.6006
Params: C=0.1, kernel=sigmoid, gamma=scale → Val Accuracy: 0.6006
Params: C=0.1, kernel=sigmoid, gamma=auto → Val Accuracy: 0.6006
Params: C=0.1, kernel=poly, gamma=scale → Val Accuracy: 0.9235
Params: C=0.1, kernel=poly, gamma=auto → Val Accuracy: 0.6006
Params: C=1, kernel=rbf, gamma=scale → Val Accuracy: 0.9491
Params: C=1, kernel=rbf, gamma=auto → Val Accuracy: 0.6006
Params: C=1, kernel=sigmoid, gamma=scale → Val Accuracy: 0.5927
Params: C=1, kernel=sigmoid, gamma=auto → Val Accuracy: 0.6006
Params: C=1, kernel=poly, gamma=scale → Val Accuracy: 0.9341
Params: C=1, kernel=poly, gamma=auto → Val Accuracy: 0.6006
Params: C=10, kernel=rbf, gamma=scale → Val Accuracy: 0.9561
Params: C=10, kernel=rbf, gamma=auto → Val Accuracy: 0.6006
Params: C=10, kernel=sigmoid, gamma=scale → Val Accuracy: 0.5201
Params: C=10, kernel=sigmoid, gamma=auto → Val Accuracy: 0.6006
Para